In [ ]:
!pip install -q pypdf
!pip install -q python-dotenv
!pip install -q llama-index
!pip install -q gradio
!pip install einops
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
pip install llama-index-llms-huggingface

In [ ]:
pip install llama-index-embeddings-instructor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 1.7 MB/s eta 0:00:00


In [ ]:
pip install llama-index-embeddings-huggingface

In [ ]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext,PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM

In [ ]:
documents = SimpleDirectoryReader("/content/Data").load_data()

In [ ]:
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext,PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM



In [ ]:

from llama_index.core.prompts.prompts import SimpleInputPrompt

system_prompt = ''' Act as Assistant. Assistant is a sales executive in finance company, responsible for selling Personal Loan to Customers.
You should not answer anything outside of the given details.

'''
# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")



llm = HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.1, "do_sample": True},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="microsoft/phi-2",
    model_name="microsoft/phi-2",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16},

)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# loads BAAI/bge-small-en
# embed_model = HuggingFaceEmbedding()

# loads BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model,

)



<ipython-input-34-6381714ad0ec>:9: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

query_engine = index.as_query_engine()

def predict(input, history):
  response = query_engine.query(input)
  return str(response)


In [ ]:

import gradio as gr

gr.ChatInterface(predict).launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://edba541c73b5c4aaf0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 501, in call_prediction
    output = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 253, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1695, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1233, in call_function
    prediction = await fn(*processed_input)
  File "/usr/local/lib/python3.10/dist-packages/gradio/utils.py", line 659, in async_wrapper
    response = await f(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/gradio/chat_interface.py", line 466, in _submit_fn
    response = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packag

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7864 <> https://edba541c73b5c4aaf0.gradio.live


In [ ]:
from inspect import cleandoc

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = "microsoft/phi-2"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float32,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True

)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from transformers import GenerationConfig, TextStreamer, pipeline

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.do_sample = True

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    streamer=streamer,
)

In [ ]:

SYSTEM_PROMPT = """
You're helpful assistant that always answers truthfully.
""".strip()


def create_prompt(prompt: str, system_prompt: str = SYSTEM_PROMPT) -> str:
    if not system_prompt:
        return cleandoc(
            f"""
        Instruct: {prompt}
        Output:
        """
        )
    return cleandoc(
        f"""
        Instruct: {system_prompt} {prompt}
        Output:
        """
    )


prompt = create_prompt("What are the pros/cons of ChatGPT vs Open Source LLMs?")
print(prompt)

Instruct: You're helpful assistant that always answers truthfully. What are the pros/cons of ChatGPT vs Open Source LLMs?
Output:


In [ ]:
data = """
- Customer can apply for Minimum loan Amount : 1,00,000 INR
- Minimum Salary for Salaried employed to apply for loan is 25,000 INR.
- Rate of interest for any loan is 18%
- Tenure : Tenure applicable is 12,24,36,48,60 Months only. Do not accept any other tenure from the customer.
- processing fees : Upto 3%
- Age: 25 To 58
"""

prompt = cleandoc(
    f"""
    Act as Assistant. Assistant is a sales executive in Bajaj Finance Ltd, responsible for selling Personal Loan to Customers.
    what are the available tenure months ?

```
{data}
```
"""
)

output = llm(create_prompt(prompt))

        Instruct: You're helpful assistant that always answers truthfully.     Act as Assistant. Assistant is a sales executive in Bajaj Finance Ltd, responsible for selling Personal Loan to Customers.
    what are the available tenure months?

```

- Customer can apply for Minimum loan Amount : 1,00,000 INR
- Minimum Salary for Salaried employed to apply for loan is 25,000 INR.
- Rate of interest for any loan is 18%
- Tenure : Tenure applicable is 12,24,36,48,60 Months only. Do not accept any other tenure from the customer.
- processing fees : Upto 3% 
- Age: 25 To 58

```
        Output:
        